## Introduction
Text processing has a long history and is known under a couple of different names including Computational Linguistics and Natural Language Processing.  in this notebook we will look at some text procesing options avaiable in the packages

* nltk
* gensim
* scikit-learn

### nltk - The Natural Language Toolkit
nltk is a very popular text processing tool for western languages.  It can be found at the [nltk site](http://www.nltk.org/).



In [5]:
import os
import nltk

nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords


# load training data
BASE_DIR = '../data'
TEXT_DATA_DIR = os.path.join(BASE_DIR, 'SpookyData')

import pandas as pd

# read the training data
df = pd.read_csv(os.path.join(TEXT_DATA_DIR, 'train.csv'))
df.head()


[nltk_data] Downloading package stopwords to /home/joe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/joe/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [6]:
# Read the text of the training examples  
corpus = df['text'].tolist()
corpus[0]

'This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall.'

In [8]:
from nltk.tokenize import word_tokenize

print(word_tokenize(corpus[0]))

['This', 'process', ',', 'however', ',', 'afforded', 'me', 'no', 'means', 'of', 'ascertaining', 'the', 'dimensions', 'of', 'my', 'dungeon', ';', 'as', 'I', 'might', 'make', 'its', 'circuit', ',', 'and', 'return', 'to', 'the', 'point', 'whence', 'I', 'set', 'out', ',', 'without', 'being', 'aware', 'of', 'the', 'fact', ';', 'so', 'perfectly', 'uniform', 'seemed', 'the', 'wall', '.']


In [15]:
# Remove english stopwords from the tokenized lists
stops = set(stopwords.words('english'))
modified_corpus = []
for sent in corpus:
    modified_sent = []
    for term in word_tokenize(sent):
        if term not in stops:
            modified_sent.append(term)
    modified_corpus.append(modified_sent)
print(modified_corpus[0])

['This', 'process', ',', 'however', ',', 'afforded', 'means', 'ascertaining', 'dimensions', 'dungeon', ';', 'I', 'might', 'make', 'circuit', ',', 'return', 'point', 'whence', 'I', 'set', ',', 'without', 'aware', 'fact', ';', 'perfectly', 'uniform', 'seemed', 'wall', '.']


In [ ]:
# create a labeled set of training features.
